In [1]:
#!pip install h2o

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

import h2o
from h2o.automl import H2OAutoML

h2o.__version__

'3.30.1.3'

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "14.0.2" 2020-07-14; OpenJDK Runtime Environment (build 14.0.2+12); OpenJDK 64-Bit Server VM (build 14.0.2+12, mixed mode)
  Starting server from /usr/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpnfocy_ea
  JVM stdout: /tmp/tmpnfocy_ea/h2o_javi_started_from_python.out
  JVM stderr: /tmp/tmpnfocy_ea/h2o_javi_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.3
H2O_cluster_version_age:,1 month and 11 days
H2O_cluster_name:,H2O_from_python_javi_2wlf59
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.898 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


# Leer datos

In [3]:
df = pd.read_csv("../data/adult.csv")
print("Filas dataset:", len(df))
df.head()

Filas dataset: 48842


,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


# Simple split

In [4]:
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=0)
print("Filas train:", len(train_df))
print("Filas valid:", len(valid_df))

Filas train: 39073
Filas valid: 9769


# Pandas dataframe -> H2OFrame

In [5]:
train = h2o.H2OFrame(train_df)
valid = h2o.H2OFrame(valid_df)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
type(train)

h2o.frame.H2OFrame

### Identify predictors and response

In [8]:
x = train.columns
y = "income"
x.remove(y)

### For binary classification, response should be a factor

In [9]:
train[y] = train[y].asfactor()
valid[y] = valid[y].asfactor()

### Run AutoML for 20 base models (limited to 1 hour max runtime by default)

In [10]:
#aml = H2OAutoML(max_models=20, seed=1)
aml = H2OAutoML(max_runtime_secs=60, seed=1)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |█████████
00:59:39.283: GLM_1_AutoML_20201109_005928 [GLM def_1] failed: java.lang.ArrayIndexOutOfBoundsException: Index 108 out of bounds for length 108

███████████████████████████████████████████████| 100%


### View the AutoML Leaderboard & Print all rows instead of default (10 rows)

In [12]:
lb = h2o.automl.get_leaderboard(aml, extra_columns='ALL')
lb.head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms
StackedEnsemble_AllModels_AutoML_20201109_005928,0.927723,0.292806,0.827902,0.166309,0.3005,0.0903002,744,0.009978
XGBoost_grid__1_AutoML_20201109_005928_model_1,0.927094,0.27878,0.826159,0.165372,0.297596,0.0885634,766,0.002606
XGBoost_grid__1_AutoML_20201109_005928_model_2,0.926758,0.280831,0.824903,0.168921,0.298158,0.0888982,440,0.001754
StackedEnsemble_BestOfFamily_AutoML_20201109_005928,0.926517,0.295088,0.825929,0.17652,0.301367,0.0908218,967,0.010952
GBM_grid__1_AutoML_20201109_005928_model_1,0.919502,0.296193,0.810398,0.186483,0.305033,0.0930449,791,0.008236
XGBoost_3_AutoML_20201109_005928,0.914775,0.309356,0.800393,0.184067,0.310711,0.0965415,244,0.001134
XGBoost_2_AutoML_20201109_005928,0.912949,0.327182,0.801585,0.181528,0.313408,0.0982248,222,0.001477
DeepLearning_grid__1_AutoML_20201109_005928_model_1,0.910293,0.315142,0.776294,0.184127,0.315822,0.0997438,448,0.004429
GBM_4_AutoML_20201109_005928,0.910252,0.389047,0.775835,0.181752,0.345921,0.119661,192,0.002036
DeepLearning_grid__2_AutoML_20201109_005928_model_1,0.90988,0.313713,0.775495,0.179142,0.316233,0.100003,792,0.006


In [13]:
type(lb)

h2o.frame.H2OFrame

# Obtener mejor modelo

In [22]:
model_names = list(aml.leaderboard['model_id'].as_data_frame().iloc[:,0])

In [30]:
best_ensemble     = h2o.get_model(model_names[0]) # aml.leader
best_single_model = h2o.get_model(model_names[1])

# Predecir cjto. de validación

In [31]:
preds = best_ensemble.predict(valid)
preds

stackedensemble prediction progress: |████████████████████████████████████| 100%


predict,<=50K,>50K
<=50K,0.93889,0.06111
>50K,0.0690943,0.930906
>50K,0.138125,0.861875
<=50K,0.919674,0.0803259
<=50K,0.966672,0.0333277
<=50K,0.965194,0.0348064
<=50K,0.960347,0.0396534
<=50K,0.84884,0.15116
<=50K,0.839788,0.160212
<=50K,0.965421,0.0345786


# Balanced accuracy = 86%

In [57]:
y_true = (preds.as_data_frame()["predict"]==">50K").values
y_pred = (preds.as_data_frame()["<=50K"] < .5).values

In [58]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_true, y_pred)

0.860096870342772